In [1]:
# If true, the WAV files will be read and their features will be saved in the CSV files
# As this is the most time consuming task, only enable it if you don't have the CSV files yet
CREATE_CSV_FILES = False

In [ ]:
# Defines the names of the CSV files
TRAIN_CSV_FILE      = "train.csv"
TEST_CSV_FILE       = "test.csv"

# TRAIN_CSV_FILE_CLose ="train_close.csv"
# TRAIN_CSV_FILE_Open  ="train_open.csv"
# TEST_CSV_FILE_CLose ="test_close.csv"
# TEST_CSV_FILE_Open  ="test_open.csv"


In [ ]:
import matplotlib.pyplot as plt
import pickle 
import numpy as np
from matplotlib import cm
import librosa
import csv
import os

def extractWavFeatures(soundFilesFolder, csvFileName,genre):
    print("The features of the files in the folder "+soundFilesFolder+" will be saved to "+csvFileName)
    # Header
    header = 'state filename chroma_stft rmse spectral_centroid spectral_bandwidth rolloff zero_crossing_rate'
    for i in range(1, 21):
        header += f' mfcc{i}'
    header += ' label'
    header = header.split()
    print('CSV Header: ', header)
    # preaparing csv file
    file = open(csvFileName, 'w', newline='')
    writer = csv.writer(file)
    writer.writerow(header)

    # looping through records
    for filename in os.listdir(soundFilesFolder):
        number  = f'{soundFilesFolder}/{filename}'
        state   =genre
        # reading records
        y, sr = librosa.load(number)
        # remove leading and trailing silence
        y, index    = librosa.effects.trim(y)

        state       = genre
        chroma_stft = librosa.feature.chroma_stft(y=y, sr=sr)
        rmse        = librosa.feature.rms(y=y)
        spec_cent   = librosa.feature.spectral_centroid(y=y, sr=sr)
        spec_bw     = librosa.feature.spectral_bandwidth(y=y, sr=sr)
        rolloff     = librosa.feature.spectral_rolloff(y=y, sr=sr)
        zcr         = librosa.feature.zero_crossing_rate(y)
        mfcc        = librosa.feature.mfcc(y=y, sr=sr)
        to_append   = f'{state} {filename} {np.mean(chroma_stft)} {np.mean(rmse)} {np.mean(spec_cent)} {np.mean(spec_bw)} {np.mean(rolloff)} {np.mean(zcr)}'
        for e in mfcc:
            to_append += f' {np.mean(e)}'
        writer.writerow(to_append.split())
    file.close()
    print("End of extractWavFeatures")

if (CREATE_CSV_FILES == True):
    # extractWavFeatures("train/open" , TRAIN_CSV_FILE_Open ,1)
    # extractWavFeatures("train/close" , TRAIN_CSV_FILE_CLose,0)

    # extractWavFeatures("test/open"  , TEST_CSV_FILE_Open ,1)
    # extractWavFeatures("test/close" , TEST_CSV_FILE_CLose,0)

    # extractWavFeatures("../data/recordings/moreSpeakersTrain", MORE_TRAIN_CSV_FILE)
    # extractWavFeatures("../data/recordings/moreSpeakersTest", MORE_TEST_CSV_FILE)
    print("CSV files are created")
else:
    print("CSV files creation is skipped")
    


In [2]:

# Importing the required packages
import pickle
import librosa
import numpy as np
import pandas as pd
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report

def prepare_testing(to_test):
    features=[]
    # reading records
    y, sr = librosa.load(to_test)
    # remove leading and trailing silence
    y, index    = librosa.effects.trim(y)

    chroma_stft = librosa.feature.chroma_stft(y=y, sr=sr)
    rmse        = librosa.feature.rms(y=y)
    spec_cent   = librosa.feature.spectral_centroid(y=y, sr=sr)
    spec_bw     = librosa.feature.spectral_bandwidth(y=y, sr=sr)
    rolloff     = librosa.feature.spectral_rolloff(y=y, sr=sr)
    zcr         = librosa.feature.zero_crossing_rate(y)
    mfcc        = librosa.feature.mfcc(y=y, sr=sr)
    to_append   = f' {np.mean(rmse)} {np.mean(spec_cent)} {np.mean(spec_bw)} {np.mean(rolloff)} {np.mean(zcr)}'
    for e in mfcc:
        to_append += f' {np.mean(e)}'
    
    features.append(to_append.split())

    
    for index in range(0,len(features[0])):
        features[0][index]=float(features[0][index])

    return features

    



  
# Function importing Dataset
def importdata():
    data = pd.read_csv('train.csv')
    data = data.drop(['filename'],axis=1)
    data = data.drop(['label'],axis=1)
    data = data.drop(['chroma_stft'],axis=1)

    # # Printing the dataswet shape
    # print ("Dataset Length: ", len(data))
    # print ("Dataset Shape: ", data.shape)
      
    # # Printing the dataset obseravtions
    # print ("Dataset: ",data.head())
    return data
  
# Function to split the dataset
def splitdataset(balance_data):
  
    # Separating the target variable
    X = balance_data.values[:, 1:]
    Y = balance_data.values[:, 0]
    # Splitting the dataset into train and test
    X_train, X_test, y_train, y_test = train_test_split( X, Y, test_size = 0.3, random_state = 100)
      
    return X, Y, X_train, X_test, y_train, y_test
      
# Function to perform training with giniIndex.
def train_using_gini(X_train, X_test, y_train):
  
    # Creating the classifier object
    clf_gini = DecisionTreeClassifier(criterion = "gini",random_state = 100,max_depth=3, min_samples_leaf=5)
    # Performing training
    clf_gini.fit(X_train, y_train)
    return clf_gini
      
# Function to perform training with entropy.
def tarin_using_entropy(X_train, X_test, y_train):

    # Decision tree with entropy
    clf_entropy = DecisionTreeClassifier(criterion = "entropy", random_state = 100,max_depth = 3, min_samples_leaf = 5)
    # Performing training
    clf_entropy.fit(X_train, y_train)
    return clf_entropy
  
  
# Function to make predictions
def prediction(X_test, clf_object):
  
    # Predicton on test with giniIndex
    # print('x_test')
    # print(X_test[0])
    y_pred = clf_object.predict(X_test)
    # print("Predicted values:")
    # print(y_pred)
    return y_pred
      
# Function to calculate accuracy
def cal_accuracy(y_test, y_pred):
      
    print("Confusion Matrix: ",confusion_matrix(y_test, y_pred))
      
    print ("Accuracy : ",accuracy_score(y_test,y_pred)*100)
      
    print("Report : ",classification_report(y_test, y_pred))
  
# Driver code
def main():

    # to_predict=[[],[]]
    to_predict=prepare_testing('k_close_2.wav')
    
    # Building Phase
    data = importdata()

    X, Y, X_train, X_test, y_train, y_test = splitdataset(data)
    clf_gini    = train_using_gini(X_train, X_test, y_train)
    clf_entropy = tarin_using_entropy(X_train, X_test, y_train)
      
    # # Operational 
    # print("Results Using Gini Index:-----------------")
      
    # # Prediction using gini
    # y_pred_gini = prediction(X_test, clf_gini)
    # cal_accuracy(y_test, y_pred_gini)
      
    print("Results Using Entropy:---------------------")
    # Prediction using entropy
    print(X_test)
    print(to_predict)
    y_pred_entropy = prediction(to_predict, clf_entropy)
    if y_pred_entropy==0:
        print('Close')
    elif y_pred_entropy==1:
        print('Open')

    # cal_accuracy(y_test, y_pred_entropy)
    model ='model.pkl'
    pickle.dump(clf_entropy,open(model,'wb'))
      
      
# Calling main function
if __name__=="__main__":
    main()

Results Using Entropy:---------------------
[[ 8.65143100e-03  1.47756177e+03  1.90946123e+03 ... -2.64368129e+00
  -7.93565798e+00 -4.12586117e+00]
 [ 1.48427427e-01  1.77548039e+03  1.82374153e+03 ... -1.14715576e+01
  -4.17657757e+00 -5.88370132e+00]
 [ 7.38329500e-03  1.46013622e+03  1.89309178e+03 ...  2.16274762e+00
  -1.79716659e+00 -1.10321054e+01]
 ...
 [ 2.21242870e-02  2.65793746e+03  2.73963809e+03 ... -1.82252884e+00
  -5.81420898e+00 -2.80799389e+00]
 [ 8.12090900e-03  1.37846442e+03  1.81837766e+03 ... -2.79363632e+00
  -5.17642116e+00 -6.52206993e+00]
 [ 7.89907100e-03  1.68077813e+03  1.94560515e+03 ...  5.99143696e+00
  -6.02419472e+00 -1.05705471e+01]]
[[0.013147185556590557, 2849.6503584460247, 2326.4879458307123, 5264.329833984375, 0.16525472005208333, -549.3910522460938, 51.08045196533203, 17.5028076171875, 2.9545342922210693, -3.0804476737976074, 3.726375102996826, -21.580549240112305, -2.282191514968872, -17.786502838134766, -9.046612739562988, -12.0225276947021